In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import torch

In [2]:
data_types_dict = {
    'time_id': 'int32',
    'investment_id': 'int16',
    "target": 'float16',
}

features = [f'f_{i}' for i in range(300)]

for f in features:
    data_types_dict[f] = 'float16'

In [3]:
data = pd.read_csv('../input/ubiquant-market-prediction/train.csv', 
                       usecols = data_types_dict.keys(),
                       dtype=data_types_dict,
                       index_col = 0, chunksize = 3141410) #3141410

In [4]:
data = next(iter(data))

/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
#plt.plot(data.groupby('time_id').count())
#plt.plot()

In [6]:
"""
time_id = 625
X_train = data.loc[:time_id-1,]
X_test = data.loc[time_id:,]
print(X_train.shape[0], X_test.shape[0])

y_train = X_train['target']
X_train.drop(['target', 'investment_id'], inplace=True, axis=1)

y_test = X_test['target']
X_test.drop(['target', 'investment_id'], inplace=True, axis=1)
print(y_train.shape[0], y_test.shape[0])
print(y_test.shape[0]/y_train.shape[0])
"""

"\ntime_id = 625\nX_train = data.loc[:time_id-1,]\nX_test = data.loc[time_id:,]\nprint(X_train.shape[0], X_test.shape[0])\n\ny_train = X_train['target']\nX_train.drop(['target', 'investment_id'], inplace=True, axis=1)\n\ny_test = X_test['target']\nX_test.drop(['target', 'investment_id'], inplace=True, axis=1)\nprint(y_train.shape[0], y_test.shape[0])\nprint(y_test.shape[0]/y_train.shape[0])\n"

In [7]:
target = data['target']
data.drop(['target', 'investment_id'], inplace=True, axis=1)

In [8]:
"""
from scipy.stats import pearsonr 
def est_score(model):
    y_pred = model.predict(X_test)
    return pearsonr(y_pred, y_test)[0]
"""

'\nfrom scipy.stats import pearsonr \ndef est_score(model):\n    y_pred = model.predict(X_test)\n    return pearsonr(y_pred, y_test)[0]\n'

In [9]:
"""
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, data['target'], test_size=0.1, stratify=data.index)

X_train.drop(['target', 'investment_id'], inplace=True, axis=1)
X_test.drop(['target', 'investment_id'], inplace=True, axis=1)
"""

"\nfrom sklearn.model_selection import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(data, data['target'], test_size=0.1, stratify=data.index)\n\nX_train.drop(['target', 'investment_id'], inplace=True, axis=1)\nX_test.drop(['target', 'investment_id'], inplace=True, axis=1)\n"

In [10]:
#del data

In [11]:
#from sklearn.linear_model import LinearRegression
#reg_model = LinearRegression(copy_X=False).fit(X_train, y_train)

In [12]:
#est_score(reg_model)

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def move_to(data, device):
    """
    moving data to device
    :param data: data to move
    :param device: device
    :return: moved data
    """

    if isinstance(data, (list, tuple)):
        return [move_to(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [14]:
def batch_loader(X, y, batch_size = 4096, equal=False):
    i = 0
    while True:
        if i + batch_size >= y.shape[0]: 
            break
            
        yield X.iloc[i:i+batch_size, :], y[i:i+batch_size]
        i += batch_size

In [15]:
def pearson_loss(x, y):
    vx = x - torch.mean(x)
    vy = y - torch.mean(y)
    loss = -torch.sum(vx*vy) / (torch.sqrt(torch.sum(vx*vx)) * torch.sqrt(torch.sum(vy*vy)))
    return loss

In [16]:
class DNN_Model(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.linear1_1 = torch.nn.Linear(300, deep_size)
        self.linear1_2 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear2_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear2_2 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear3_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear3_2 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear4_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear4_2 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear5_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear5_2 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear6_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear6_2 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear7_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear7_2 = torch.nn.Linear(deep_size, 1)
        
        self.dout = torch.nn.Dropout(p=0.3)
        self.act = torch.nn.SiLU()
    def fit(self, X, y, iters, batch_size = 4096, subset = 0.8, silent=False, plot=False, val=False, X_val=None, y_val=None):
        if val:
            tensor_train_val = move_to(torch.FloatTensor(X_val.values), device)
            tensor_y_val = move_to(torch.FloatTensor(y_val.values), device)
        scores_train = []
        scores_val = []
        
        opt = torch.optim.Adam(self.parameters(), lr=1e-3)
        
        for epoch in range(iters):
            i = 0
            #learning cycle
            for X_batch, y_batch in batch_loader(X, y):
                self.train()
                self.zero_grad()
                X_tensor = move_to(torch.FloatTensor(X_batch.values), device)
                y_tensor = move_to(torch.FloatTensor(y_batch.values), device)
                loss = pearson_loss(self.tensor_forward(X_tensor).view(-1), y_tensor)
                loss.backward()
                opt.step()
                i+=batch_size
                
            #prints
            if val:
                self.eval()
                val_loss = pearson_loss(self.tensor_forward(tensor_train_val).view(-1), tensor_y_val)
                scores_val.append(float(val_loss.detach()))
            scores_train.append(float(loss.detach()))
            
            if not silent:
                print(f'epoch {epoch} finished with {loss} loss')
                if val:
                    print(f'val loss {val_loss}')
                
            
        if not silent:
            print(f'Finished with {loss} loss')
        if plot:
            if val:
                sns.lineplot(x=range(iters), y=scores_train, label='train')
                sns.lineplot(x=range(iters), y=scores_val, label='val')
            else:
                sns.lineplot(scores_train)
            plt.plot()
                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        tensor_X = self.act(self.linear1_1(X_tensor))
        res_1 = self.act(self.linear1_2(tensor_X))
        
        tensor_X = self.act(self.linear2_1(res_1))
        tensor_X = self.dout(tensor_X)
        res_2 = self.act(self.linear2_2(tensor_X))
        
        tensor_X = self.act(self.linear3_1(res_2))
        tensor_X = self.dout(tensor_X)
        res_3 = self.act(self.linear3_2(tensor_X))
        
        tensor_X = self.act(self.linear4_1(res_3))
        tensor_X = self.act(self.linear4_2(tensor_X))
        
        tensor_X = self.act(self.linear5_1(tensor_X + res_3))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear5_2(tensor_X))
        
        tensor_X = self.act(self.linear6_1(tensor_X + res_2))
        tensor_X = self.dout(tensor_X)
        tensor_X = self.act(self.linear6_2(tensor_X))
        
        tensor_X = self.act(self.linear7_1(tensor_X + res_1))
        tensor_X = self.linear7_2(tensor_X)
        return tensor_X 

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [17]:
class DNN_Model_1(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        act_1 = torch.nn.SiLU
        act_2 = torch.nn.ReLU
        self.model = self.model = torch.nn.Sequential(
            torch.nn.Linear(300, deep_size),
            act_1(),
            
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Linear(deep_size, deep_size),
            act_1(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, 1)
            
        )
    def fit(self, X, y, iters, batch_size = 4096, subset = 0.8, silent=False, plot=False, val=False, X_val=None, y_val=None):
        if val:
            tensor_train_val = move_to(torch.FloatTensor(X_val.values), device)
            tensor_y_val = move_to(torch.FloatTensor(y_val.values), device)
        scores_train = []
        scores_val = []
        
        opt = torch.optim.Adam(self.parameters(), lr=5e-4)
        
        for epoch in range(iters):
            i = 0
            #learning cycle
            while i + batch_size < y.shape[0]: 
                self.train()
                self.zero_grad()
                X_tensor = move_to(torch.FloatTensor(X.iloc[i:i+batch_size, :].values), device)
                y_tensor = move_to(torch.FloatTensor(y[i:i+batch_size].values), device)
                loss = pearson_loss(self.tensor_forward(X_tensor).view(-1), y_tensor)
                loss.backward()
                opt.step()
                i+=batch_size
                
            #prints
            if val:
                self.eval()
                val_loss = pearson_loss(self.tensor_forward(tensor_train_val).view(-1), tensor_y_val)
                scores_val.append(float(val_loss.detach()))
            scores_train.append(float(loss.detach()))
            
            if not silent:
                print(f'epoch {epoch} finished with {loss} loss')
                if val:
                    print(f'val loss {val_loss}')
                
            
        if not silent:
            print(f'Finished with {loss} loss')
        if plot:
            if val:
                sns.lineplot(x=range(iters), y=scores_train, label='train')
                sns.lineplot(x=range(iters), y=scores_val, label='val')
            else:
                sns.lineplot(scores_train)
            plt.plot()
                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        return self.model(X_tensor)

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [18]:
class DNN_Model_2(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(300, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=0.3),
            
            torch.nn.Linear(deep_size, deep_size),
            torch.nn.SiLU(),
            torch.nn.Linear(deep_size, 1)
        )

    def fit(self, X, y, iters, batch_size = 4096, subset = 0.8, silent=False, plot=False, val=False, X_val=None, y_val=None):
        self = move_to(self, device)
        if val:
            tensor_train_val = move_to(torch.FloatTensor(X_val.values), device)
            tensor_y_val = move_to(torch.FloatTensor(y_val.values), device)
        scores_train = []
        scores_val = []
        
        opt = torch.optim.Adam(self.parameters(), lr=5e-4)
        
        for epoch in range(iters):
            i = 0
            #learning cycle
            while i + batch_size < y.shape[0]: 
                self.train()
                self.zero_grad()
                X_tensor = move_to(torch.FloatTensor(X.iloc[i:i+batch_size, :].values), device)
                y_tensor = move_to(torch.FloatTensor(y[i:i+batch_size].values), device)
                loss = pearson_loss(self.tensor_forward(X_tensor).view(-1), y_tensor)
                loss.backward()
                opt.step()
                i+=batch_size
                
            #prints
            if val:
                self.eval()
                val_loss = pearson_loss(self.tensor_forward(tensor_train_val).view(-1), tensor_y_val)
                scores_val.append(float(val_loss.detach()))
            scores_train.append(float(loss.detach()))
            
            if not silent:
                print(f'epoch {epoch} finished with {loss} loss')
                if val:
                    print(f'val loss {val_loss}')
                
            
        if not silent:
            print(f'Finished with {loss} loss')
        if plot:
            if val:
                sns.lineplot(x=range(iters), y=scores_train, label='train')
                sns.lineplot(x=range(iters), y=scores_val, label='val')
            else:
                sns.lineplot(scores_train)
            plt.plot()
                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        return self.model(X_tensor)

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [19]:
class DNN_Model_3(torch.nn.Module):
    def __init__(self, deep_size):
        super().__init__()
        self.linear1 = torch.nn.Linear(300, deep_size)
        
        self.linear2_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear2_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear2_3 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear3_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear3_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear3_3 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear4_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear4_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear4_3 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear5_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear5_2 = torch.nn.Linear(deep_size, deep_size)
        self.linear5_3 = torch.nn.Linear(deep_size, deep_size)
        
        self.linear6_1 = torch.nn.Linear(deep_size, deep_size)
        self.linear6_2 = torch.nn.Linear(deep_size, 1)
        
        self.act = torch.nn.SiLU()
        self.dout = torch.nn.Dropout(p=0.5)

    def fit(self, X, y, iters, batch_size = 4096, subset = 0.8, silent=False, plot=False, val=False, X_val=None, y_val=None):
        self = move_to(self, device)
        if val:
            tensor_train_val = move_to(torch.FloatTensor(X_val.values), device)
            tensor_y_val = move_to(torch.FloatTensor(y_val.values), device)
        scores_train = []
        scores_val = []
        
        opt = torch.optim.Adam(self.parameters(), lr=1e-3)
        
        for epoch in range(iters):
            i = 0
            #learning cycle
            while i + batch_size < y.shape[0]: 
                self.train()
                self.zero_grad()
                X_tensor = move_to(torch.FloatTensor(X.iloc[i:i+batch_size, :].values), device)
                y_tensor = move_to(torch.FloatTensor(y[i:i+batch_size].values), device)
                loss = pearson_loss(self.tensor_forward(X_tensor).view(-1), y_tensor)
                loss.backward()
                opt.step()
                i+=batch_size
                
            #prints
            if val:
                self.eval()
                val_loss = pearson_loss(self.tensor_forward(tensor_train_val).view(-1), tensor_y_val)
                scores_val.append(float(val_loss.detach()))
            scores_train.append(float(loss.detach()))
            
            if not silent:
                print(f'epoch {epoch} finished with {loss} loss')
                if val:
                    print(f'val loss {val_loss}')
                
            
        if not silent:
            print(f'Finished with {loss} loss')
        if plot:
            if val:
                sns.lineplot(x=range(iters), y=scores_train, label='train')
                sns.lineplot(x=range(iters), y=scores_val, label='val')
            else:
                sns.lineplot(scores_train)
            plt.plot()
                
    def tensor_forward(self, X_tensor):
        X_tensor = move_to(X_tensor, device)
        
        X_tensor = self.act(self.linear1(X_tensor))
        
        X_tensor = self.act(self.linear2_1(X_tensor))
        X_tensor = self.act(self.linear2_2(X_tensor))
        X_tensor = self.dout(X_tensor)
        res_2 = self.act(self.linear2_3(X_tensor))
        
        
        #res_2 = X_tensor
        
        X_tensor = self.act(self.linear3_1(res_2))
        X_tensor = self.act(self.linear3_2(X_tensor))
        X_tensor = self.dout(X_tensor)
        res_3 = self.act(self.linear3_3(X_tensor))
        
        
        #res_3 = X_tensor
        
        X_tensor = self.act(self.linear4_1(res_2 + res_3))
        X_tensor = self.act(self.linear4_2(X_tensor))
        X_tensor = self.dout(X_tensor)
        res_4 = self.act(self.linear4_3(X_tensor))
        
        res_4 = X_tensor

        X_tensor = self.act(self.linear5_1(res_3 + res_4))
        X_tensor = self.act(self.linear5_2(X_tensor))
        X_tensor = self.dout(X_tensor)
        X_tensor = self.act(self.linear5_3(X_tensor))
        
        X_tensor = self.act(self.linear6_1(X_tensor + res_4))
        X_tensor = self.linear6_2(X_tensor)
        
        return X_tensor
        

    def forward(self, X):
        tensor_X = torch.FloatTensor(X.values)
        return self.tensor_forward(tensor_X)

In [20]:
#написать subsample еще раз (также попробовать booststrap)
#написать еще пару моделей, желательно как можно более разных
#из большого количества моделей выбрать топ лучших (на полном датасете!)
#продумать как можно (например с помощью матрицы корреляции) стакать сети в одной модели чтобы увеличить скор
#поработать с бустами, попробовать их пообъединять

In [21]:
class StackedDNN:
    def __init__(self, n, model, min_deep=30, max_deep=40, min_iter=10, max_iter=15):
        self.nets = [model(np.random.randint(min_deep, max_deep)) for i in range(n)]
        for net in self.nets:
            net = move_to(net, device)
        self.min_iter=min_iter
        self.max_iter=max_iter
    
    def fit(self, X, y, batch_size=4096, subset=0.8):
        for i in range(len(self.nets)):
            self.nets[i].fit(X, y, np.random.randint(self.min_iter, self.max_iter), batch_size, subset, silent=True, plot=False, val=False, X_val=None, y_val=None)
            print(f'{i+1}/{len(self.nets)} fitted')
        
    def predict(self, X):
        y_pred = np.ndarray([len(self.nets), X.shape[0]])
        for i in range(len(self.nets)):
            self.nets[i].eval()
            y_pred[i] = np.array(self.nets[i].forward(X).detach().view(-1).cpu())
        y_pred /= len(self.nets)
        return np.sum(y_pred, axis=0)

In [22]:
#0   0.16049979767507658
#1   0.16195405253013023
#2   0.16213173833206326
#3   0.16256979869460275
#4

In [23]:
nn_model = StackedDNN(25, DNN_Model, min_iter=5, max_iter=8, min_deep=90, max_deep=110)

#nn_model = StackedDNN(1, DNN_Model, min_iter=6, max_iter=11, min_deep=280, max_deep=301)

In [24]:
nn_model.fit(data, target)

1/25 fitted
2/25 fitted
3/25 fitted
4/25 fitted
5/25 fitted
6/25 fitted
7/25 fitted
8/25 fitted
9/25 fitted
10/25 fitted
11/25 fitted
12/25 fitted
13/25 fitted
14/25 fitted
15/25 fitted
16/25 fitted
17/25 fitted
18/25 fitted
19/25 fitted
20/25 fitted
21/25 fitted
22/25 fitted
23/25 fitted
24/25 fitted
25/25 fitted


In [25]:
#res = nn_model.predict(X_test)
#pearsonr(res, y_test)[0]

In [26]:
nn_model_1 = StackedDNN(25, DNN_Model_1, min_iter=11, max_iter=13, min_deep=40, max_deep=41)

#nn_model = StackedDNN(10, min_iter=12, max_iter=16, min_deep=280, max_deep=301)

In [27]:
nn_model_1.fit(data, target)

1/25 fitted
2/25 fitted
3/25 fitted
4/25 fitted
5/25 fitted
6/25 fitted
7/25 fitted
8/25 fitted
9/25 fitted
10/25 fitted
11/25 fitted
12/25 fitted
13/25 fitted
14/25 fitted
15/25 fitted
16/25 fitted
17/25 fitted
18/25 fitted
19/25 fitted
20/25 fitted
21/25 fitted
22/25 fitted
23/25 fitted
24/25 fitted
25/25 fitted


In [28]:
#res_1 = nn_model_1.predict(X_test)
#pearsonr(res_1, y_test)[0]

In [29]:
nn_model_2 = StackedDNN(25, DNN_Model_2, min_iter=9, max_iter=11, min_deep=100, max_deep=101)
nn_model_2.fit(data, target)

#StackedDNN(1, DNN_Model_2, min_iter=10, max_iter=12, min_deep=100, max_deep=101)

1/25 fitted
2/25 fitted
3/25 fitted
4/25 fitted
5/25 fitted
6/25 fitted
7/25 fitted
8/25 fitted
9/25 fitted
10/25 fitted
11/25 fitted
12/25 fitted
13/25 fitted
14/25 fitted
15/25 fitted
16/25 fitted
17/25 fitted
18/25 fitted
19/25 fitted
20/25 fitted
21/25 fitted
22/25 fitted
23/25 fitted
24/25 fitted
25/25 fitted


In [30]:
#res_2 = nn_model_2.predict(X_test)
#pearsonr(res_2, y_test)[0]

In [31]:
nn_model_3 = StackedDNN(25, DNN_Model_3, min_iter=6, max_iter=8, min_deep=90, max_deep=110)

nn_model_3.fit(data, target)

#StackedDNN(1, DNN_Model_2, min_iter=10, max_iter=12, min_deep=100, max_deep=101)

1/25 fitted
2/25 fitted
3/25 fitted
4/25 fitted
5/25 fitted
6/25 fitted
7/25 fitted
8/25 fitted
9/25 fitted
10/25 fitted
11/25 fitted
12/25 fitted
13/25 fitted
14/25 fitted
15/25 fitted
16/25 fitted
17/25 fitted
18/25 fitted
19/25 fitted
20/25 fitted
21/25 fitted
22/25 fitted
23/25 fitted
24/25 fitted
25/25 fitted


In [32]:
#res_3 = nn_model_3.predict(X_test)
#pearsonr(res_3, y_test)[0]

In [33]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=10000, task_type='GPU', devices='0', silent=True).fit(data, target)

In [34]:
#est_score(model)

In [35]:
#res_boost = model.predict(X_test)

In [36]:
"""
best = 0
for i in range(10000):
    a = np.random.rand(5)
    a /= np.sum(a)
    #print(pearsonr(res_1, y_test)[0])
    #print(pearsonr(res_2, y_test)[0])
    #print(pearsonr(res_3, y_test)[0])
    total = pearsonr(a[0]*res + a[1]*res_1 + a[2]*res_2 + a[3]*res_3 + a[4]*res_boost, y_test)[0]
    if total > best:
        best = total
        print(a, best)
"""

'\nbest = 0\nfor i in range(10000):\n    a = np.random.rand(5)\n    a /= np.sum(a)\n    #print(pearsonr(res_1, y_test)[0])\n    #print(pearsonr(res_2, y_test)[0])\n    #print(pearsonr(res_3, y_test)[0])\n    total = pearsonr(a[0]*res + a[1]*res_1 + a[2]*res_2 + a[3]*res_3 + a[4]*res_boost, y_test)[0]\n    if total > best:\n        best = total\n        print(a, best)\n'

In [37]:
#[0.001, 0.337, 0.116, 0.01, 0.536] - coefs

In [38]:
#pearsonr(0.001*res + 0.337*res_1 + 0.116*a[2]*res_2 + 0.01*a[3]*res_3 + 0.536*res_boost, y_test)[0]

In [39]:
"""
a = 0.8
print(pearsonr(res, y_test)[0])
print(pearsonr(res_boost, y_test)[0])
pearsonr(a*res_boost+(1-a)*res, y_test)[0]
"""

'\na = 0.8\nprint(pearsonr(res, y_test)[0])\nprint(pearsonr(res_boost, y_test)[0])\npearsonr(a*res_boost+(1-a)*res, y_test)[0]\n'

In [40]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    res = nn_model.predict(test_df[features])
    res_1 = nn_model_1.predict(test_df[features])
    res_2 = nn_model_2.predict(test_df[features])
    res_3 = nn_model_3.predict(test_df[features])
    res_boost = model.predict(test_df[features])
    #[0.001, 0.337, 0.116, 0.01, 0.536]
    test_df['target']  = 0.001*res + 0.337*res_1 + 0.116*res_2 + 0.01*res_3 + 0.536*res_boost
    env.predict(test_df[['row_id','target']])

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
